Import libraries.

In [23]:
import numpy as np
import pandas as pd
import requests 

Get the HTML of the Wiki page, convert into a table with help of read_html, remove cells with a borough that is Not assigned.

In [24]:
url=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df_r = pd.read_html(url.content, header=0)[0]
df_n = df_r[df_r.Borough != 'Not assigned']
df_n.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Find whether there is a "Not assigned" in Neighborhood, and replaceit with the value of Borough.

In [25]:
df_n.loc[df_n.Neighborhood == 'Not assigned']
df_n.Neighborhood.replace('Not assigned', df_n.Borough, inplace=True)

C:\Users\sabaa\anaconda3\lib\site-packages\pandas\core\generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [26]:
df_toronto= df_n.groupby(['Postal Code', 'Borough'])['Neighborhood'].apply(lambda x: ', '.join(x))
df_toronto = df_toronto.reset_index()
df_toronto.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [27]:
df_toronto.shape

(103, 3)

Get the latitude and the longitude coordinates of each neighborhood.
Used the csv file, because of some problems with geocoder

In [28]:
url = 'http://cocl.us/Geospatial_data'
df_geo=pd.read_csv(url)
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [29]:
df_geo.shape

(103, 3)

Both dataframes has the same shape.Now we should join new colums to our data.

In [30]:
df_toronto = df_toronto.join(df_geo.set_index('Postal Code'), on='Postal Code')

In [31]:
df_toronto.head()


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
